## Product Sentiment Data - Learning Rates - 2nd Attempt

Data (public domain): https://data.world/crowdflower/brands-and-product-emotions

Notebook code based on IMDB notebook from bert-sklearn/other_examples

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import csv
import re
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from ftfy import fix_text
 
from bert_sklearn import BertClassifier
from bert_sklearn import load_model

print(os.getcwd())

DATAFILE = "./data/judge-cleaned-up.csv"

/Users/joseph.porter/Data/nas2019/NAS2019


In [3]:
# Prep Data

def cleanup(txt):
    return fix_text(txt)
    
converters = {'tweet_text': cleanup}
    
raw_data = pd.read_csv(DATAFILE, converters=converters, encoding='unicode_escape')
raw_data.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
6,,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


In [4]:
## Transform columns
## ONLY RUN THIS CELL ONCE!!!

# Add columns to make the labels usable by the model
# tweet_text => text
# Positive / No emotion / Negative => 1, 0, -1
# Product: Apple stuff, Google stuff, NaN => Apple, Google, ''

def clean_text(txt):
    return txt
raw_data.insert(1, "text", np.vectorize(clean_text)(raw_data['tweet_text']))

def create_labels(sentiment):
    if sentiment.startswith('Positive'):
        return 1
    if sentiment.startswith('Negative'):
        return -1
    return 0
raw_data.insert(3, 'label', np.vectorize(create_labels)(raw_data['is_there_an_emotion_directed_at_a_brand_or_product']))

def get_company(product):
    if pd.isnull(product):
        return ''
    if 'iPad' in product or 'iPhone' in product or 'Apple' in product:
        return 'Apple'
    if 'Google' in product or 'Android' in product:
        return 'Google'
    return ''
raw_data.insert(2, 'company', np.vectorize(get_company)(raw_data['emotion_in_tweet_is_directed_at']))
raw_data.head(10)

,tweet_text,text,company,emotion_in_tweet_is_directed_at,label,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,Apple,iPhone,-1,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,@jessedee Know about @fludapp ? Awesome iPad/i...,Apple,iPad or iPhone App,1,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,@swonderlin Can not wait for #iPad 2 also. The...,Apple,iPad,1,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,@sxsw I hope this year's festival isn't as cra...,Apple,iPad or iPhone App,-1,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Google,1,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,@teachntech00 New iPad Apps For #SpeechTherapy...,,NaN,0,No emotion toward brand or product
6,,,,NaN,0,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...","#SXSW is just starting, #CTIA is around the co...",Google,Android,1,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,Beautifully smart and simple idea RT @madebyma...,Apple,iPad or iPhone App,1,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Counting down the days to #sxsw plus strong Ca...,Apple,Apple,1,Positive emotion


In [5]:
# Last Data Preparation Step
# Clean up characters and pull out columns of interest

def clean(text):
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r"\"", "", text)       
    return text

data = raw_data.filter(['text', 'company', 'label'], axis=1)
data['text'] = data['text'].transform(clean)

In [6]:
# Split into training and test data

msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]
print('Training data size: ' + str(train.shape))
print('Test data size: ' + str(test.shape))

Training data size: (7327, 3)
Test data size: (1766, 3)


In [7]:
train[:1].values

array([['.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.',
        'Apple', -1]], dtype=object)

As you can see, each review is much longer than a sentence or two. The Google AI BERT models were trained on sequences of max length 512. Lets look at the performance for max_seq_length equal to  128, 256, and 512.

### max_seq_length = 128

In [8]:
## Set up data for the classifier

train = train.sample(500)
test = test.sample(300)

print("Train data size: %d "%(len(train)))
print("Test data size: %d "%(len(test)))

X_train = train['text']
y_train = train['label']

X_test = test['text']
y_test = test['label']

Train data size: 500 
Test data size: 300 


In [11]:
## Create the model

model = BertClassifier(bert_model='bert-base-uncased', label_list=[-1,0,1])
model.max_seq_length = 128
model.learning_rate = 2e-05
model.train_batch_size = 16
model.epochs = 4

print(model)


Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=[-1, 0, 1], learning_rate=2e-05,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=16,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)


In [12]:
%%time
## Train the model using our data (this could take a while)

model.fit(X_train, y_train)

accy = model.score(X_test, y_test)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 450, validation data size: 50



Training  : 100%|██████████| 29/29 [06:19<00:00, 13.07s/it, loss=0.955]

Validating: 100%|██████████| 7/7 [00:10<00:00,  1.52s/it]

Epoch 1, Train loss: 0.9549, Val loss: 0.8764, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [06:31<00:00, 13.51s/it, loss=0.86]

Validating: 100%|██████████| 7/7 [00:09<00:00,  1.41s/it]

Epoch 2, Train loss: 0.8602, Val loss: 0.8756, Val accy: 58.00%


...

Epoch 3, Train loss: 0.8539, Val loss: 0.8561, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [05:58<00:00, 12.37s/it, loss=0.79]

Validating: 100%|██████████| 7/7 [00:09<00:00,  1.37s/it]

Epoch 4, Train loss: 0.7902, Val loss: 0.8211, Val accy: 58.00%




Testing: 100%|██████████| 38/38 [00:57<00:00,  1.52s/it]


Loss: 0.8150, Accuracy: 60.67%
CPU times: user 1h 3min 50s, sys: 5min 39s, total: 1h 9min 30s
Wall time: 26min 26s


In [17]:
y_pred = model.predict(X_test)
report = classification_report(y_pred, y_test, labels=[-1,0,1])
print(report)


Predicting: 100%|██████████| 38/38 [01:05<00:00,  1.72s/it]

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       1.00      0.61      0.76       300
           1       0.00      0.00      0.00         0

    accuracy                           0.61       300
   macro avg       0.33      0.20      0.25       300
weighted avg       1.00      0.61      0.76       300




/anaconda3/envs/bert/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


## Increase the Learning Rate 10X (to speed things up)

In [13]:
## Create the model

model2 = BertClassifier(bert_model='bert-base-uncased', label_list=[-1,0,1])
model2.max_seq_length = 128
model2.learning_rate = 2e-04
model2.train_batch_size = 16
model2.epochs = 4

print(model2)

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=[-1, 0, 1], learning_rate=0.0002,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=16,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)


In [14]:
%%time
## Train the model using our data (this could take a while)

model2.fit(X_train, y_train)

accy = model2.score(X_test, y_test)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 450, validation data size: 50



Training  : 100%|██████████| 29/29 [06:05<00:00, 12.60s/it, loss=0.955]

Validating: 100%|██████████| 7/7 [00:10<00:00,  1.50s/it]

Epoch 1, Train loss: 0.9551, Val loss: 0.8929, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [06:10<00:00, 12.79s/it, loss=0.93]

Validating: 100%|██████████| 7/7 [00:10<00:00,  1.54s/it]

Epoch 2, Train loss: 0.9297, Val loss: 0.8889, Val accy: 58.00%


...

Epoch 3, Train loss: 0.8879, Val loss: 0.8923, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [06:10<00:00, 12.78s/it, loss=0.864]

Validating: 100%|██████████| 7/7 [00:09<00:00,  1.41s/it]

Epoch 4, Train loss: 0.8642, Val loss: 0.8801, Val accy: 58.00%




Testing: 100%|██████████| 38/38 [00:58<00:00,  1.54s/it]


Loss: 0.8541, Accuracy: 60.67%
CPU times: user 1h 4min 38s, sys: 5min 45s, total: 1h 10min 24s
Wall time: 26min 26s


In [18]:
y_pred = model2.predict(X_test)
report2 = classification_report(y_pred, y_test, labels=[-1,0,1])
print(report2)


Predicting: 100%|██████████| 38/38 [01:05<00:00,  1.71s/it]

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       1.00      0.61      0.76       300
           1       0.00      0.00      0.00         0

    accuracy                           0.61       300
   macro avg       0.33      0.20      0.25       300
weighted avg       1.00      0.61      0.76       300



## Decrease the Learning Rate /10 (to get better accuracy)

In [15]:
model3 = BertClassifier(bert_model='bert-base-uncased', label_list=[-1,0,1])
model3.max_seq_length = 128
model3.learning_rate = 2e-04
model3.train_batch_size = 16
model3.epochs = 4

print(model3)

Building sklearn text classifier...
BertClassifier(bert_config_json=None, bert_model='bert-base-uncased',
               bert_vocab=None, do_lower_case=None, epochs=4, eval_batch_size=8,
               fp16=False, from_tf=False, gradient_accumulation_steps=1,
               ignore_label=None, label_list=[-1, 0, 1], learning_rate=0.0002,
               local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
               max_seq_length=128, num_mlp_hiddens=500, num_mlp_layers=0,
               random_state=42, restore_file=None, train_batch_size=16,
               use_cuda=True, validation_fraction=0.1, warmup_proportion=0.1)


In [16]:
%%time
## Train the model using our data (this could take a while)

model3.fit(X_train, y_train)

accy = model3.score(X_test, y_test)

Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 450, validation data size: 50



Training  : 100%|██████████| 29/29 [06:05<00:00, 12.60s/it, loss=0.955]

Validating: 100%|██████████| 7/7 [00:09<00:00,  1.41s/it]

Epoch 1, Train loss: 0.9551, Val loss: 0.8929, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [06:08<00:00, 12.71s/it, loss=0.93]

Validating: 100%|██████████| 7/7 [00:10<00:00,  1.45s/it]

Epoch 2, Train loss: 0.9297, Val loss: 0.8889, Val accy: 58.00%


...

Epoch 3, Train loss: 0.8879, Val loss: 0.8923, Val accy: 58.00%




Training  : 100%|██████████| 29/29 [06:14<00:00, 12.93s/it, loss=0.864]

Validating: 100%|██████████| 7/7 [00:10<00:00,  1.55s/it]

Epoch 4, Train loss: 0.8642, Val loss: 0.8801, Val accy: 58.00%




Testing: 100%|██████████| 38/38 [01:01<00:00,  1.62s/it]


Loss: 0.8541, Accuracy: 60.67%
CPU times: user 1h 4min 48s, sys: 5min 47s, total: 1h 10min 35s
Wall time: 26min 28s


In [19]:
y_pred = model3.predict(X_test)
report3 = classification_report(y_pred, y_test, labels=[-1,0,1])
print(report3)


Predicting: 100%|██████████| 38/38 [01:06<00:00,  1.75s/it]

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       1.00      0.61      0.76       300
           1       0.00      0.00      0.00         0

    accuracy                           0.61       300
   macro avg       0.33      0.20      0.25       300
weighted avg       1.00      0.61      0.76       300

